使用bert进行情感分析，以商品评价数据集为例
数据集来源：https://github.com/SophonPlus/ChineseNlpCorpus


## Step1 导包

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset,load_from_disk

## Step2 加载数据集

In [139]:
dataset = load_dataset("csv", data_files="./JD.com_comments.csv", split='train')
# dataset=dataset.shuffle(10065).select(range ( 100000 )) # 选取部分数据

In [163]:
dataset[0]

{'userId': 29792,
 'productId': 345003,
 'rating': 5,
 'timestamp': 1368720000,
 'title': '路由器很好',
 'comment': '这款路由器用着感觉不错 感觉是好久没卖出去了吧,那么多的灰尘'}

## Step3 数据集预处理
在数据集中，rating,title,comment分别表示评分，评价标题，评价内容。我们仅使用rating和comment
剔除数据集中rating或者comment为空的数据

In [141]:
dataset=dataset.filter( lambda item:  item['rating'] is not None and item['comment'] is not None)

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

加载分词器，构造编码函数，用来将汉字编码成为张量

In [143]:
# 加载模型的分词器
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

In [144]:
def process_function(examples):
    '''
    数据处理函数
    rating在数据集中共可取1,2,3,4,5。我们最终的结果与rating的对应关系为：
    好评(0)：4,5
    中性(1)：3
    差评(2):1,2
    '''
    content = examples['comment']
    tokenized_examples = tokenizer(content, max_length=128, truncation=True, padding="max_length")

    rates = [int(ra) for ra in examples["rating"]]
    tokenized_examples["labels"] = [0 if rate > 3 else 2 if rate < 3 else 1 for rate in rates]
    return tokenized_examples

In [145]:
# 进行数据处理
tokenized_datasets = dataset.map(process_function, batched=True, remove_columns=dataset.column_names)
tokenized_datasets

Map:   0%|          | 0/56585 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 56585
})

In [32]:
tokenized_datasets.save_to_disk('./waimai_data')

In [165]:
# 从本地加载数据
tokenized_datasets=load_from_disk('./waimai_data')
tokenized_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 4081167
})

## Step4 划分数据集

把整个数据集按比例划分为训练集和测试集

In [166]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3673050
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 408117
    })
})

## Step5 创建模型

In [149]:
# 因为我们是一个多分类问题，需要指定num_labels
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3",num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step6 创建评估函数

In [150]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metirc = evaluate.load("f1")


In [151]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metirc.compute(predictions=predictions, references=labels,average='micro')
    acc.update(f1)
    return acc

## Step7 创建TrainingArguments

In [155]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=2048,  # 训练时的batch_size
                               per_device_eval_batch_size=2048,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               num_train_epochs=10,              # 训练轮数
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型
train_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_s

## Step8 创建Trainer

In [156]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model,
                  args=train_args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["test"],
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

## Step9 模型训练

In [157]:
trainer.train()

/root/anaconda3/envs/tf/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=250, training_loss=0.43612158012390134, metrics={'train_runtime': 711.8635, 'train_samples_per_second': 715.39, 'train_steps_per_second': 0.351, 'total_flos': 8548939048135680.0, 'train_loss': 0.43612158012390134, 'epoch': 10.0})

## Step10 模型预测

In [ ]:
id2_label = { 1: "中性",2:"差评！",0: "好评！"}
model.eval()

In [127]:
# 从chekcpoints中加载模型
# model= AutoModelForSequenceClassification.from_pretrained('./checkpoints/checkpoint-30')

In [159]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [169]:
print(pipe("还不错，老板送小礼品了"))
print(pipe("一般般，勉强能用"))
print(pipe("太垃圾了，跟图片上差远了"))

[{'label': '好评！', 'score': 0.9717923998832703}]
[{'label': '中性', 'score': 0.5955721139907837}]
[{'label': '差评！', 'score': 0.8747568726539612}]
